In [16]:
import spcm
from spcm import units
import numpy as np

# card : spcm.Card

ip = 'TCPIP::192.168.1.83::INSTR'

card = spcm.Card(ip)

card.open(ip)

# setup card for DDS
card.card_mode(spcm.SPC_REP_STD_DDS)

# Setup the channels
channels = spcm.Channels(card)
print(len(channels))
channels.enable(True)
channels.output_load(50 * units.ohm)
channels.amp(.29 * units.V)
card.write_setup()

# trigger mode
trigger = spcm.Trigger(card)
trigger.or_mask(spcm.SPC_TMASK_EXT0) # disable default software trigger
trigger.ext0_mode(spcm.SPC_TM_POS) # positive edge
trigger.ext0_level0(1.5 * units.V) # Trigger level is 1.5 V (1500 mV)
trigger.ext0_coupling(spcm.COUPLING_DC) # set DC coupling
card.write_setup()

# Setup DDS functionality
dds = spcm.DDS(card, channels=channels)
dds.reset()

dds.trg_src(spcm.SPCM_DDS_TRG_SRC_CARD)

core_list = [hex(2**n) for n in range(20)]

# core_list = [spcm.SPCM_DDS_CORE8,  spcm.SPCM_DDS_CORE9,  spcm.SPCM_DDS_CORE10, spcm.SPCM_DDS_CORE11, spcm.SPCM_DDS_CORE20]

# dds.cores_on_channel(1, *core_list)
card.start(spcm.M2CMD_CARD_ENABLETRIGGER)

2


In [23]:
def split_array(f_split = 2.4e6, f_width_single_side = 4.e6, n_per_side = 4):
    f_center = 74.8e6
    # array1 = np.linspace( f_center-f_split/2-f_width_single_side ,f_center-f_split/2,n_per_side)
    array1 = f_center - f_split/2 - np.linspace( f_width_single_side , 0. ,n_per_side)
    # array2 = np.linspace( f_center+f_split/2+f_width_single_side ,f_center+f_split/2,n_per_side)
    array2 = f_center + f_split/2 + np.linspace( 0. , f_width_single_side ,n_per_side)
    # array1 = np.linspace(70.5e6,72.5e6,n)
    # array2 = np.linspace(77.5e6,79.5e6,n)
    return np.concatenate([array1,array2])

def compute_tweezer_1064_phases(n_tweezers,amplitudes):
    phase_tweezer_array = np.empty([n_tweezers])
    for tweezer_idx in range(n_tweezers):
        if tweezer_idx == 0:
            phase_tweezer_array[0] = 360
        else:
            phase_ij = 0
            for j in range(1,tweezer_idx):
                phase_ij = phase_ij + 2*np.pi*(tweezer_idx - j)*amplitudes[tweezer_idx]
            phase_i = (phase_ij % 2*np.pi) * 360
            phase_tweezer_array[tweezer_idx] = phase_i
    return phase_tweezer_array

# f_list = split_array(f_split=2.4e6)
# f_list = [76.e6,77.e6,78.e6,79.0e6,80.e6]
# f_list = np.linspace(76.e6,80.e6,5)
# f_list = np.linspace(76.e6,80.e6,4)
f_list = [73.9e6,77.e6]
# f_list = [70.98e6,79.16e6]
# f_list = [75.5e6]
# f_list = [76.e6,77.e6,78.e6,79.0e6,80.e6]
# f_list1 = np.linspace(76.e6,79.e6,5)
# f_list2 = np.linspace(71.e6,74.e6,5)
# f_list = [72.e6, 76.e6, 80.e6]
# f_list = np.concatenate([f_list1,f_list2])

# f_list = [70.8e6]
# a_list = [0.36393715]

# f_list = [75.e6]
# a_list = [0.99999]

# f_list = [76.e6]
# a_list = [0.19369715]

# f_list = [80.e6]
# a_list = [0.10826336]

n_tweezers = len(f_list)

# a_list1 = np.linspace(1/(2*(n_tweezers)),1/(2*(n_tweezers)),int(n_tweezers/2))
# a_list2 = 2*a_list1

# a_list = np.concatenate([a_list1,a_list2])
# a_list = np.linspace(1/((n_tweezers)),1/((n_tweezers)),int(n_tweezers))
# a_list = [.1,.1,.1,.05,.1,.35]
# a_list = [0.22, 0.17, 0.15, 0.16, 0.17]
a_list = [.43,.5]
# a_list = [.76]
# a_list = [.8,.65,.5,.18]
# a_list = [0.22, 0.17, 0.15, 0.16, 0.17]
# a_list = [0.2, 0.2, 0.2, 0.2, 0.2]
# a_list = [0.3524204, 0.3235297, 0.18756762, 0.10483738]
# a_list = [2 , 0.9258065, 0.31613, 0.46451613]

f_list = np.array(f_list)
a_list = np.array(a_list)

# idx = [0,1,2,3]
# a_list = a_list[idx]
# f_list = f_list[idx]

phases = compute_tweezer_1064_phases(len(f_list),a_list)

# def normalize_alist(alist):
#     sum = np.sum(alist)
#     return alist/sum
# a_list = normalize_alist(a_list)

for tweezer_idx in range(len(core_list)):
    if tweezer_idx < len(f_list):
        dds[tweezer_idx].amp(a_list[tweezer_idx])
        dds[tweezer_idx].freq(f_list[tweezer_idx])
        dds[tweezer_idx].phase(phases[tweezer_idx])
    # else:
    #     dds[tweezer_idx].amp(0.)
dds.exec_at_trg()
dds.write_to_card()
trigger.force()

# Start command including enable of trigger engine

# input("Press Enter to Exit")

a_list

array([0.43, 0.5 ])

In [25]:
# card.close(card._handle)
card.stop()
card.close(card._handle)

OSError: exception: access violation writing 0x0000000000000024